In [1]:
import tensorflow as tf
import pandas as pd
import random
from __future__ import division
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.HDFStore("train.h5", "r").get("train")

In [3]:
_mean = data.median(axis=0)
data.fillna(0 ,inplace=True);

In [4]:
train, test = train_test_split(data, test_size=0.25, random_state=8, shuffle= False)
y_train = train.y
X_train = train.drop(['y','id'], axis=1)
y_test = test.y
X_test = test.drop(['y','id'], axis=1)

In [5]:
def r_score(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    return (np.sign(r2)*np.sqrt(np.abs(r2)))

In [6]:
epochs=500
batch_size=50000
feature_size=X_train.shape[1]

In [7]:
x = tf.placeholder("float", [batch_size, feature_size])
y = tf.placeholder("float", [batch_size])
weights = {
    'h1': tf.get_variable("h1", shape=[feature_size, 512],
           initializer=tf.contrib.layers.xavier_initializer()),
    'h2': tf.get_variable("h2", shape=[512, 1024],
           initializer=tf.contrib.layers.xavier_initializer()),
    'h3': tf.get_variable("h3", shape=[1024, 1024],
           initializer=tf.contrib.layers.xavier_initializer()),
    'h4': tf.get_variable("h4", shape=[1024, 512],
           initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable("out", shape=[512, 1],
           initializer=tf.contrib.layers.xavier_initializer()),
}
weights1 = {
    'h1': tf.Variable(tf.truncated_normal([feature_size, 512],mean=0.0001, stddev=0.00001)),
    'h2': tf.Variable(tf.truncated_normal([512, 1024],mean=0.0001, stddev=0.00001)),
    'h3': tf.Variable(tf.truncated_normal([1024, 1024],mean=0.0001, stddev=0.00001)),
    'h4': tf.Variable(tf.truncated_normal([1024, 512],mean=0.0001, stddev=0.00001)),
    'out': tf.Variable(tf.truncated_normal([512, 1],mean=0.0001, stddev=0.00001)),
}

biases = {
    'b1': tf.Variable(tf.random_normal([512], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([1024], 0, 0.1)),
    'b3': tf.Variable(tf.random_normal([1024], 0, 0.1)),
    'b4': tf.Variable(tf.random_normal([512], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([1], 0, 0.1))
}

In [8]:
def deep_net(x, weights, biases,keep_prob=0.5):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3 = tf.nn.dropout(layer_3, keep_prob)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    layer_4 = tf.nn.dropout(layer_4, keep_prob)

    # Output layer with linear activation
    out_layer = tf.add(tf.matmul(layer_4, weights['out']), biases['out'])
    return out_layer

In [35]:
pred =   tf.transpose(deep_net(x, weights1, biases))
sub=pred-y
sq=tf.square(pred-y)
top=tf.nn.top_k(pred,k=1)
top_pred=pred[0,top.indices[0,0]]
top_y=x[top.indices[0,0]]
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost)

In [38]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    random.seed(0)
    for epoch in range(epochs):
        selected=random.sample(list(X_train.index), batch_size)
        x_batch=X_train.loc[selected]
        y_batch=y_train.loc[selected]
        _, c, p ,s ,sq1, t, tp, ty= sess.run([optimizer, cost, pred, sub, sq, top, top_pred, top_y], feed_dict={
                x: x_batch,
                y: y_batch})
        print (epoch, ' : ', c)
        print ('p: ' ,p.max())
        print ('sub: ', s)
        print ('sq: ', sq1)
        print ('top: ', t)
        print ('top_pred: ', tp)
        print ('top_y: ', ty)
        #print ('y: ', y_batch)
        #print (y_batch.iloc[0])
        #print (p[0,0]-s[0,0])

0  :  7.18682e+14
p:  3.03407e+09
sub:  [[ 0.05295882  0.04315594  0.07402678 ...,  0.04431923  0.02908303
   0.04711113]]
sq:  [[ 0.00280464  0.00186244  0.00547996 ...,  0.00196419  0.00084582
   0.00221946]]
top:  TopKV2(values=array([[  3.03406541e+09]], dtype=float32), indices=array([[1088]], dtype=int32))
top_pred:  3.03407e+09
top_y:  [  6.90000000e+01   2.30193093e-01   1.78094745e-01   1.30029947e-01
   1.06319153e+00  -2.78694704e-02   5.84487915e-01   0.00000000e+00
  -2.44378909e-01  -9.52463523e-02   7.69388258e-01   0.00000000e+00
  -1.45378947e-01  -9.86815467e-02   5.43925047e-01  -5.55893732e-03
  -2.64740705e-01   7.43460432e-02  -3.34155420e-03  -2.31472566e-03
  -9.87527445e-02   6.61572665e-02   1.03888511e+18   2.43388817e-01
   4.21616495e-01  -9.23467726e-02   5.87104023e-01  -3.35120261e-01
   1.28000602e-01   1.69497699e-01   4.30726945e-01   0.00000000e+00
   5.58913112e-01  -1.16537377e-01  -3.31707507e-01  -6.13646507e-02
  -1.07387789e-01   5.83009608e-03 

4  :  7.98111e+12
p:  4.45313e+08
sub:  [[ 0.06321454  0.06625331  0.04991517 ...,  0.04911997  0.06310488
   0.0527348 ]]
sq:  [[ 0.00399608  0.0043895   0.00249152 ...,  0.00241277  0.00398223
   0.00278096]]
top:  TopKV2(values=array([[  4.45313216e+08]], dtype=float32), indices=array([[28012]], dtype=int32))
top_pred:  4.45313e+08
top_y:  [  8.90000000e+01   1.89553708e-01   1.30421117e-01   1.10273384e-01
   1.75518346e+00  -2.90103145e-02   4.38468516e-01   0.00000000e+00
  -2.59091526e-01  -8.61019939e-02   7.50873804e-01   0.00000000e+00
  -1.72222406e-01  -9.44575518e-02   5.98226547e-01  -7.91377481e-03
  -2.84896106e-01   7.91346580e-02  -3.69563466e-03   5.30253630e-03
  -1.10175282e-01   6.88051209e-02   1.04027894e+18  -6.19916506e-02
   4.21616495e-01  -9.42843258e-02   5.91142476e-01  -3.41283619e-01
   9.20009911e-02   1.70049205e-01   1.15658537e-01   0.00000000e+00
   3.27818662e-01  -1.02352694e-01  -2.09361598e-01  -7.68271787e-03
  -9.82266292e-02   6.04655547e-03

8  :  1.94373e+11
p:  9.21265e+07
sub:  [[ 0.06867503  0.03958198  0.04311135 ...,  0.06413639  0.05502611
   0.06545524]]
sq:  [[ 0.00471626  0.00156673  0.00185859 ...,  0.00411348  0.00302787
   0.00428439]]
top:  TopKV2(values=array([[ 92126488.]], dtype=float32), indices=array([[35728]], dtype=int32))
top_pred:  9.21265e+07
top_y:  [  1.19000000e+02   1.69675931e-01   1.07130729e-01   1.00602768e-01
   1.88344324e+00  -2.95637771e-02   3.69173169e-01   0.00000000e+00
  -2.49270350e-01  -8.16494599e-02   7.41857767e-01   0.00000000e+00
  -1.63509980e-01  -9.23983008e-02   6.08223975e-01  -9.06260777e-03
  -2.70745426e-01   8.14741924e-02  -3.89581686e-03   9.01817810e-03
  -1.05727121e-01   7.00953975e-02   1.04027894e+18  -1.83229670e-01
   4.21616495e-01  -9.49645936e-02   6.01158023e-01  -3.44292223e-01
   7.43653700e-02   1.63513660e-01  -3.82609554e-02   0.00000000e+00
   2.14924768e-01  -9.54567119e-02  -1.49544328e-01   2.55232751e-02
  -9.37895402e-02   6.18753675e-03   9.6

12  :  3.88072e+10
p:  2.52177e+07
sub:  [[ 0.05033325  0.05826472  0.04958124 ...,  0.04915069  0.1056343
   0.0506997 ]]
sq:  [[ 0.00253344  0.00339478  0.0024583  ...,  0.00241579  0.01115861
   0.00257046]]
top:  TopKV2(values=array([[ 25217676.]], dtype=float32), indices=array([[32290]], dtype=int32))
top_pred:  2.52177e+07
top_y:  [  1.24000000e+02   1.69578731e-01   1.07016832e-01   1.00555472e-01
   1.87137163e+00  -2.95664836e-02   4.13402081e-01   0.00000000e+00
  -2.49641776e-01  -8.16276893e-02   7.41813660e-01   0.00000000e+00
  -1.64004788e-01  -9.23882276e-02   6.09974980e-01  -9.06822551e-03
  -2.71267354e-01   8.14856365e-02  -3.89679568e-03   9.03634820e-03
  -1.05952509e-01   7.01017082e-02   1.04027894e+18  -1.84390500e-01
   4.21616495e-01  -9.49744433e-02   6.01568043e-01  -3.44306946e-01
   7.42791295e-02   1.62247047e-01  -3.90136689e-02   0.00000000e+00
   2.14372680e-01  -9.54229906e-02  -1.49251804e-01   2.55150627e-02
  -9.37678367e-02   6.18822640e-03   6.3

16  :  1.20611e+10
p:  9.71302e+06
sub:  [[ 0.05057277  0.01265786  0.03368643 ...,  0.05142781  0.04630983
   0.03366852]]
sq:  [[ 0.0025576   0.00016022  0.00113478 ...,  0.00264482  0.0021446
   0.00113357]]
top:  TopKV2(values=array([[ 9713020.]], dtype=float32), indices=array([[40562]], dtype=int32))
top_pred:  9.71302e+06
top_y:  [  1.31000000e+02   1.69536456e-01   1.06967300e-01   1.00534908e-01
   1.86612165e+00  -2.95676608e-02   4.32637691e-01   0.00000000e+00
  -2.49803320e-01  -8.16182196e-02   7.41794467e-01   0.00000000e+00
  -1.64219990e-01  -9.23838541e-02   6.10736489e-01  -9.07066930e-03
  -2.71494359e-01   8.14906135e-02  -3.89722153e-03   9.04425140e-03
  -1.06050536e-01   7.01044500e-02   1.04027894e+18  -1.84895352e-01
   4.21616495e-01  -9.49787349e-02   6.01746321e-01  -3.44313353e-01
   7.42416158e-02   1.61696181e-01  -3.93410362e-02   0.00000000e+00
   2.14132577e-01  -9.54083204e-02  -1.49124578e-01   2.55114920e-02
  -9.37584043e-02   6.18852628e-03   4.94

20  :  1.42988e+09
p:  3.91982e+06
sub:  [[ 0.04973482  0.05008327  0.04864465 ...,  0.072009    0.0676752
   0.04697445]]
sq:  [[ 0.00247355  0.00250833  0.0023663  ...,  0.0051853   0.00457993
   0.0022066 ]]
top:  TopKV2(values=array([[ 3919821.25]], dtype=float32), indices=array([[20172]], dtype=int32))
top_pred:  3.91982e+06
top_y:  [  1.07000000e+02   1.70468360e-01   1.08059205e-01   1.00988284e-01
   1.94496143e+00  -2.95417141e-02   3.37935418e-01   0.00000000e+00
  -2.53842026e-01  -8.18269625e-02   7.42217183e-01   0.00000000e+00
  -1.69213042e-01  -9.24803913e-02   6.04672909e-01  -9.01680905e-03
  -2.77208120e-01   8.13809261e-02  -3.88783659e-03   8.87005311e-03
  -1.08367383e-01   7.00439587e-02   1.04027894e+18  -1.85430065e-01
   4.21616495e-01  -9.50025171e-02   5.97815990e-01  -3.44172299e-01
   7.50684142e-02   1.66141316e-01  -3.21249403e-02   0.00000000e+00
   2.19425291e-01  -9.57316235e-02  -1.51928946e-01   2.24993769e-02
  -9.39664245e-02   6.18191669e-03   1.

KeyboardInterrupt: 